In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_groq import ChatGroq #groq install


In [3]:
#load dot env
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY= os.getenv("GROQ_API_KEY")

In [ ]:
llm = ChatGroq(temperature=0.5, groq_api_key=KEY, model_name="mixtral-8x7b-32768")